In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics.pairwise import cosine_similarity
import random
import torch.nn.functional as F
import os
import numpy as np
import psycopg2
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from psycopg2.extras import DictCursor
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [45]:
# save to user_data
connection = psycopg2.connect(host="localhost", user="root", port=5432, database="W9sV6cL2dX", password="E5rG7tY3fH")
cursor = connection.cursor(cursor_factory=DictCursor)

In [46]:

select_query = "SELECT ts.user_id, ts.item_id, ts.time_spent, u.*, ie.* \
FROM time_spent AS ts \
JOIN users AS u ON ts.user_id = u.id \
JOIN item_embeddings AS ie ON ts.item_id = ie.item_id;"
cursor.execute(select_query)
data = cursor.fetchall()


In [47]:
data_col = ['user_id','item_id', 'time_spent','id' ,'gender', 'country', 'age', 'item_id2','embedding','time_stamp']
data_df = pd.DataFrame(data, columns=data_col)
data_df = data_df.drop(columns=['id', 'time_stamp', 'item_id2'])
data_df[:3]

,user_id,item_id,time_spent,gender,country,age,embedding
0,46763,ccc300bb-12bd-4e22-98dc-10527a54b57d,0,female,BayesianBourg,18.0,"[-1.1809731721878052, -0.7340627312660217, -0...."
1,38208,c8e5623c-40a1-44a5-b64a-49c2c26507e9,0,female,BayesianBourg,34.0,"[-1.1892468929290771, 0.1926315873861313, 1.13..."
2,42734,c8e5623c-40a1-44a5-b64a-49c2c26507e9,0,female,Neuropolis,27.0,"[-1.1892468929290771, 0.1926315873861313, 1.13..."


In [48]:
countries = list(set(data_df.country))
genders = list(set(data_df.gender))
gender_encoder = LabelEncoder()
gender_encoder.fit(genders)
country_encoder = LabelEncoder()
country_encoder.fit(countries)
age_scaler = MinMaxScaler()
time_scaler = MinMaxScaler()

data_df['gender_encode'] = gender_encoder.transform(data_df['gender'])
data_df['country_encode'] = country_encoder.transform(data_df['country'])
data_df['age_normalized'] = age_scaler.fit_transform(data_df[['age']])
data_df['time_normalized'] = age_scaler.fit_transform(data_df[['time_spent']])
data_df = data_df.dropna()
print(data_df.shape)
data_df[:3]

(28060, 11)


,user_id,item_id,time_spent,gender,country,age,embedding,gender_encode,country_encode,age_normalized,time_normalized
0,46763,ccc300bb-12bd-4e22-98dc-10527a54b57d,0,female,BayesianBourg,18.0,"[-1.1809731721878052, -0.7340627312660217, -0....",0,2,0.178947,0.0
1,38208,c8e5623c-40a1-44a5-b64a-49c2c26507e9,0,female,BayesianBourg,34.0,"[-1.1892468929290771, 0.1926315873861313, 1.13...",0,2,0.347368,0.0
2,42734,c8e5623c-40a1-44a5-b64a-49c2c26507e9,0,female,Neuropolis,27.0,"[-1.1892468929290771, 0.1926315873861313, 1.13...",0,5,0.273684,0.0


In [49]:
data_df[:3]

,user_id,item_id,time_spent,gender,country,age,embedding,gender_encode,country_encode,age_normalized,time_normalized
0,46763,ccc300bb-12bd-4e22-98dc-10527a54b57d,0,female,BayesianBourg,18.0,"[-1.1809731721878052, -0.7340627312660217, -0....",0,2,0.178947,0.0
1,38208,c8e5623c-40a1-44a5-b64a-49c2c26507e9,0,female,BayesianBourg,34.0,"[-1.1892468929290771, 0.1926315873861313, 1.13...",0,2,0.347368,0.0
2,42734,c8e5623c-40a1-44a5-b64a-49c2c26507e9,0,female,Neuropolis,27.0,"[-1.1892468929290771, 0.1926315873861313, 1.13...",0,5,0.273684,0.0


In [59]:
user_features_df = data_df[['user_id', 'gender_encode', 'country_encode', 'age_normalized']]
user_features_df.to_csv('user_features_df.csv')

In [50]:
user_features = data_df[['gender_encode', 'country_encode', 'age_normalized']]
item_features = data_df.apply(lambda row: row['embedding'], axis=1)
targets = data_df['time_normalized']

In [52]:
user_features = torch.tensor(user_features.values, dtype=torch.float32)
item_features = [torch.tensor(item, dtype=torch.float32) for item in item_features]
targets = torch.tensor(targets.values, dtype=torch.float32)

In [53]:
user_features.shape, len(item_features)

(torch.Size([28060, 3]), 28060)

In [54]:
type(user_features), type(item_features[1]), type(targets)

(torch.Tensor, torch.Tensor, torch.Tensor)

In [55]:
def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed) #for CPU
    torch.cuda.manual_seed(seed) #for GPU
    torch.backends.cudnn.deterministic = True


# Two-tower model architecture
class TwoTowerNetwork(nn.Module):
    def __init__(self, user_input_dim, item_input_dim, output_dim):
        super(TwoTowerNetwork, self).__init__()

        hidden_dim = 128  # example value, adjust based on your requirement

        # User tower
        self.user_tower = nn.Sequential(
            nn.Linear(user_input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

        # Item tower
        self.item_tower = nn.Sequential(
            nn.Linear(item_input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )


    def forward(self, user_input, item_input):
        # User tower
        user_output = self.user_tower(user_input)

        # Item tower
        item_output = self.item_tower(item_input)

        # Normalize the embeddings (this is necessary for cosine similarity)
        user_output = F.normalize(user_output, dim=1)
        item_output = F.normalize(item_output, dim=1)
        
        return user_output, item_output
    

# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, user_features, item_features, targets):
        self.user_features = user_features
        self.item_features = item_features
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return self.user_features[idx], self.item_features[idx], self.targets[idx]

In [56]:
# Hyperparameters
user_feature_dim = 3 # age, F/M, country
item_feature_dim = 1000 # item_id, item_type, item_features
embedding_dim = 100
batch_size = 4
learning_rate = 0.001
num_epochs = 10
seed_everything(seed=666)

In [57]:
# Create the two-tower model
model = TwoTowerNetwork(user_feature_dim, item_feature_dim, embedding_dim)

# Create the custom dataset and data loader
dataset = CustomDataset(user_features, item_features, targets)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [58]:
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for user_feats, item_feats, targets in data_loader:
        model.train()
        optimizer.zero_grad()
        
        user_embeds, item_embeds = model(user_feats, item_feats)
    
        cos_sim = nn.functional.cosine_similarity(user_embeds, item_embeds)
        prob = torch.sigmoid(cos_sim.unsqueeze(1))
        loss = criterion(prob, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader)}")

/root/miniconda3/envs/tiny/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10, Loss: 0.06145546868289486
Epoch 2/10, Loss: 0.06132980184251189
Epoch 3/10, Loss: 0.061328608342999004
Epoch 4/10, Loss: 0.06132832756279274
Epoch 5/10, Loss: 0.061328226976237354


KeyboardInterrupt: 

In [ ]:
save_path = '/root/tinier-world/notebooks/testbooks/models/'
torch.save(model.state_dict(), save_path + 'two_tower_model_ver1.pth')

In [ ]:
save_path = '/root/tinier-world/notebooks/testbooks/models/'
loaded_model = TwoTowerNetwork(user_feature_dim, item_feature_dim, embedding_dim)
loaded_model.load_state_dict(torch.load(save_path + 'two_tower_model_ver1.pth'))
loaded_model.eval()

TwoTowerNetwork(
  (user_tower): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=100, bias=True)
  )
  (item_tower): Sequential(
    (0): Linear(in_features=1000, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=100, bias=True)
  )
)